In [1]:
import os
import sys
import pickle as pkl
from argparse import ArgumentParser
from copy import deepcopy
from os.path import join as oj
import numpy as np
import pickle
import random
import configparser
import torch
from torch import nn
import torch.utils.data
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import csv
import optuna

sys.path.insert(0, "../1D CNN")
import utils
import models
import data_fns
import my_eval


random.seed(0)
np.random.seed()

config = configparser.ConfigParser()
config.read("../config.ini")
torch.backends.cudnn.deterministic = True

def get_args():
    parser = ArgumentParser(description="Functional group analysis")

    parser.add_argument("--exp_name", type=str, default="")
    parser.add_argument("--batch_size", type=int, default=128)
    parser.add_argument("--num_epochs", type=int, default=1000)
    parser.add_argument("--seed", type=int, default=0)

    parser.add_argument("--num_conv", type=int, default=16)
    parser.add_argument("--patience", type=int, default=20)
    ret_args, unknown = parser.parse_known_args()
    return ret_args


args = get_args()


# data = np.load("data.npy")
# labels = np.load('labels.npy')

# filters = (("state", "gas"), ("yunits", "ABSORBANCE"), ("xunits", "1/CM"))
# data, labels = utils.fixed_domain_filter(
#     irdata, irindex, filters
# )  # Filter points from the dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()


file_path = "TS_freq_array_0208_new.csv"
save_path = "C:\\Users\\aethe\\1D CNN"
# Initialize arrays
y_labels = []
x_TS = []
fish_num = []
final_labels = []

# Read the CSV file line by line
with open(file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    
    next(csv_reader)
    
    # Extract the first column and construct rows as arrays
    for row in csv_reader:
        fish_num.append(row[0])
        y_labels.append(row[-2])
        final_labels.append([row[-2],row[-1]])
        x_TS.append(row[2:-2])


x_TS = np.array(x_TS, dtype = float)

print(x_TS.shape) # (32954, 249)

means = np.mean(x_TS, axis=0)
std_devs = np.std(x_TS, axis=0)
x_TS = (x_TS - means) / std_devs  # Standardizing data


# Contruct data and labels
data = x_TS[:, None]
labels = np.array(y_labels, dtype=int)
final_labels = np.array(final_labels, dtype=int)

# When spliting the dataset, we first split based on each fish to aviod data leakage
# Create a mapping dictionary from fish_num to indices

fish_num_to_index = {fish_num: index for index, fish_num in enumerate(np.unique(np.array(fish_num)))}

# Use the mapping to convert the fish_num to indices
fish_ind = np.array([fish_num_to_index[num] for num in fish_num])


train_idxs_by_fish, val_idxs_by_fish, test_idxs_by_fish = data_fns.get_split(len(np.unique(fish_ind)), seed=23)

# Checn index split by fish
print(train_idxs_by_fish)
print(val_idxs_by_fish)
print(test_idxs_by_fish)

# This function takes in the indices that split by individual fish
# It then returns the indices split by each time ping correspond to the individual fish
def construct_data_index_by_fish_idxs(idxs_split_by_fish, fish_ind):
    indx = np.array([], dtype=int)  # Ensure the array is of integer type
    for split_fish_ind in idxs_split_by_fish:
        # Ensure split_fish_ind is an integer before using it as an index
        split_fish_ind = int(split_fish_ind)
        indx_current_fish = np.where(fish_ind == split_fish_ind)[0]
        indx = np.append(indx, indx_current_fish)

    return indx



# Indices that we use in the model
train_idxs = construct_data_index_by_fish_idxs(train_idxs_by_fish, fish_ind)
val_idxs = construct_data_index_by_fish_idxs(val_idxs_by_fish, fish_ind)
test_idxs = construct_data_index_by_fish_idxs(test_idxs_by_fish, fish_ind)



print(train_idxs.shape) # (22925,)
print(val_idxs.shape) # (4662,)
print(test_idxs.shape) # (5367,)

def balance_classes_by_indices(indices, labels, n_samples_per_class):
    unique_classes = np.unique(labels[indices])
    balanced_indices = np.array([], dtype=int)

    for cls in unique_classes:
        class_indices = indices[labels[indices] == cls]
        if len(class_indices) > n_samples_per_class:
            class_indices = np.random.choice(class_indices, n_samples_per_class, replace=False)
        else:
            class_indices = np.random.choice(class_indices, n_samples_per_class, replace=True)
        balanced_indices = np.append(balanced_indices, class_indices)

    np.random.shuffle(balanced_indices)  # Shuffle to mix classes
    return balanced_indices

# Apply balancing to each split
balanced_train_idxs = balance_classes_by_indices(train_idxs, labels, n_samples_per_class=10029)
balanced_val_idxs = balance_classes_by_indices(val_idxs, labels, n_samples_per_class=1000)
balanced_test_idxs = balance_classes_by_indices(test_idxs, labels, n_samples_per_class=2000)


#%%
# num_classes = labels.shape[1]
num_classes = 2
# train_idxs, val_idxs, test_idxs = data_fns.get_split(len(data), seed=42)

# scale data
# weights = labels[train_idxs].mean(axis=0)
# anti_weights = 1 - weights

# mult_weights = weights * anti_weights
# weights /= mult_weights
# anti_weights /= mult_weights

# weights = torch.tensor(weights).to(device)
# anti_weights = torch.tensor(anti_weights).to(device)


#%%
# create datasets in torch
torch.manual_seed(args.seed)
train_dataset = TensorDataset(
    *[torch.Tensor(input) for input in [data[balanced_train_idxs], final_labels[balanced_train_idxs]]],
)
val_dataset = TensorDataset(
    *[torch.Tensor(input) for input in [data[balanced_val_idxs], final_labels[balanced_val_idxs]]],
)

test_dataset = TensorDataset(
    *[torch.Tensor(input) for input in [data[balanced_test_idxs], final_labels[balanced_test_idxs]]],
)




train_loader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    shuffle=True,
)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=True)
print(num_classes)



model = models.FGANet(
    num_input=data.shape[2],
    num_output=2,
    conv_channels=args.num_conv,
    num_in_channels=data.shape[1],
    stride=1,
).to(device)






Using device: cpu

(32954, 249)
[14 18 17 32 13 33 20  8 36 28 29 10 44  3 22 48 30 35 24 23  4 42  7  1
 49 41 45 15 46 16 34 37  0  5 21]
[11  2 43  6 25 26 39]
[27 12 31 47  9 40 38 19]
(23208,)
(5182,)
(4564,)
2


In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from copy import deepcopy

def train(model, train_loader, val_loader, device, num_epochs=1000, batch_size=128,max_patience=20):
    optimizer = optim.Adam(model.parameters(),weight_decay=0.001)
    # optimizer = optim.Adam(model.parameters(),)

    training_loss = []
    test_losses = []
    test_accs = []
    
    best_test_loss = 500000
    best_weights = None
    cur_patience = 0

    for epoch in range(num_epochs):
        model.train()
        tr_loss = 0
        for _, (data_cur, labels_cur) in enumerate(train_loader):
            data_cur, labels_cur = data_cur.to(device), labels_cur.to(device)
            optimizer.zero_grad()

            y_pred = model(data_cur).squeeze()  # Ensure y_pred is of the right shape
            cur_loss = nn.functional.binary_cross_entropy(
                    y_pred, labels_cur, reduction="mean"
                )  # Ensure labels_cur is float
            # l2_lambda = 0.001
            # l2_norm = sum(p.pow(2.0).sum() for p in model.features[-5].parameters())
            # (cur_loss+l2_norm).backward()
            cur_loss.backward()
            tr_loss += cur_loss.item()                   
            optimizer.step()

        tr_loss /= len(train_loader)
        training_loss.append(tr_loss)
        # Validation loop
        model.eval()
        test_loss = 0
        test_acc = 0
        with torch.no_grad():
            for _, (data_cur, labels_cur) in enumerate(test_loader):
                data_cur, labels_cur = data_cur.to(device), labels_cur.to(device)
                y_pred = model(data_cur).squeeze()  # Ensure y_pred is of the right shape

                cur_loss =nn.functional.binary_cross_entropy(y_pred, labels_cur , reduction="mean")  # Ensure labels_cur is float
                test_loss += cur_loss.item()

                # Use the highest probability
                y_pred_classes = y_pred.argmax(dim=1)  # Convert probabilities to class indices [0 or 1]
                labels_classes = labels_cur.argmax(dim=1)  # Convert one-hot labels to class indices [0 or 1]
                test_corr = (y_pred_classes == labels_classes).sum().item()
                test_acc += test_corr

        test_acc /= len(test_loader.dataset)
        test_loss /= len(test_loader)

        test_losses.append(test_loss)
        test_accs.append(test_acc)

        print(f"Epoch: {epoch+1}, TrLoss: {tr_loss:.5f}, testLoss: {test_loss:.5f}, testAcc: {test_acc:.2f}")

        # Early stopping condition
        if test_loss < best_test_loss:
            best_weights = deepcopy(model.state_dict())
            cur_patience = 0
            best_test_loss = test_loss
            
        else:
            cur_patience += 1

        if cur_patience > max_patience:
            print("Early stopping due to no improvement in validation accuracy")
            break
    
    print("Training finished")

    model.load_state_dict(best_weights)
    np.random.seed()

    file_name = "".join([str(np.random.choice(10)) for x in range(10)])

    results = {}
    results["filename"] = file_name
    for arg in vars(args):
        if arg != "save_path":
            results[str(arg)] = getattr(args, arg)
    results["train_losses"] = training_loss
    results["test_losses"] = test_losses
    results["test_acc"] = test_accs
    results["best_test_loss"] = best_test_loss
    results["test_auc"] = my_eval.calculate_roc_auc_score(
        model, device, data[balanced_test_idxs], final_labels[balanced_test_idxs], batch_size
    )
    print("Test AUC: ", results["test_auc"])

    pkl.dump(results, open(os.path.join(save_path, file_name + ".pkl"), "wb"))
    torch.save(model.state_dict(), oj(save_path, file_name + ".pt"))
    print("Saved model and results")

    return results



if __name__ == "__main__":
    train(model, train_loader, test_loader, device,
        num_epochs=args.num_epochs,
        max_patience=args.patience,
        batch_size=args.batch_size,
    )

Epoch: 1, TrLoss: 0.57113, testLoss: 0.68929, testAcc: 0.66
Epoch: 2, TrLoss: 0.47610, testLoss: 0.71133, testAcc: 0.67
Epoch: 3, TrLoss: 0.43326, testLoss: 0.71750, testAcc: 0.67
Epoch: 4, TrLoss: 0.40285, testLoss: 0.58487, testAcc: 0.75
Epoch: 5, TrLoss: 0.37669, testLoss: 0.54746, testAcc: 0.78
Epoch: 6, TrLoss: 0.36592, testLoss: 0.61009, testAcc: 0.75
Epoch: 7, TrLoss: 0.34329, testLoss: 0.53002, testAcc: 0.80
Epoch: 8, TrLoss: 0.33527, testLoss: 0.55759, testAcc: 0.79
Epoch: 9, TrLoss: 0.32160, testLoss: 0.54687, testAcc: 0.79
Epoch: 10, TrLoss: 0.31466, testLoss: 0.62984, testAcc: 0.78
Epoch: 11, TrLoss: 0.30135, testLoss: 0.54832, testAcc: 0.78
Epoch: 12, TrLoss: 0.29476, testLoss: 0.57158, testAcc: 0.81
Epoch: 13, TrLoss: 0.28930, testLoss: 0.57540, testAcc: 0.81
Epoch: 14, TrLoss: 0.28231, testLoss: 0.52758, testAcc: 0.84
Epoch: 15, TrLoss: 0.27499, testLoss: 0.60337, testAcc: 0.78
Epoch: 16, TrLoss: 0.26959, testLoss: 0.54467, testAcc: 0.80
Epoch: 17, TrLoss: 0.26839, testL

In [12]:
def objective(trial):
    # Suggest hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256])
    num_epochs = 100  # Reduced for faster trials
    max_patience = trial.suggest_int('max_patience', 5, 20)

    # Initialize your model (adjust as necessary)
    
    # Use the suggested learning rate
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=0.001)


    best_val_loss = 50000
    cur_patience = 0

    for epoch in range(num_epochs):
        model.train()
        tr_loss = 0
        for _, (data_cur, labels_cur) in enumerate(train_loader):
            data_cur, labels_cur = data_cur.to(device), labels_cur.to(device)
            optimizer.zero_grad()

            y_pred = model(data_cur).squeeze()
            cur_loss = nn.functional.binary_cross_entropy(y_pred, labels_cur.float(), reduction="mean")
            cur_loss.backward()
            optimizer.step()

            tr_loss += cur_loss.item()

        tr_loss /= len(train_loader)
        
        model.eval()
        val_loss = 0
        val_acc = 0
        with torch.no_grad():
            for _, (data_cur, labels_cur) in enumerate(val_loader):
                data_cur, labels_cur = data_cur.to(device), labels_cur.to(device)
                y_pred = model(data_cur).squeeze()

                cur_loss = nn.functional.binary_cross_entropy(y_pred, labels_cur.float(), reduction="mean")
                val_loss += cur_loss.item()

                # predicted = (torch.sigmoid(y_pred) > 0.5).long()
                # val_corr = (predicted == labels_cur).sum()
                # val_acc += val_corr.item()
                y_pred_classes = y_pred.argmax(dim=1)  # Convert probabilities to class indices [0 or 1]
                labels_classes = labels_cur.argmax(dim=1)  # Convert one-hot labels to class indices [0 or 1]
                val_corr = (y_pred_classes == labels_classes).sum().item()
                val_acc += val_corr

        val_acc /= len(val_loader.dataset)
        val_loss /= len(val_loader)

        # Report the validation loss to optuna
        trial.report(val_loss, epoch)

        # Early stopping condition
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            cur_patience = 0
        else:
            cur_patience += 1
            if cur_patience > max_patience:
                break


    return best_val_loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)  # Adjust the number of trials

# Best trial result
print("Best trial:")
trial = study.best_trial
print(f"  Value (Best Validation Accuracy): {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-03-20 01:26:48,411] A new study created in memory with name: no-name-5d0a7219-8318-4c68-b473-5169569ec622
[I 2024-03-20 01:27:26,252] Trial 0 finished with value: 0.3628803137689829 and parameters: {'lr': 0.0009339960914009376, 'batch_size': 256, 'max_patience': 7}. Best is trial 0 with value: 0.3628803137689829.
[I 2024-03-20 01:28:21,862] Trial 1 finished with value: 0.3921430818736553 and parameters: {'lr': 0.00012938388425188987, 'batch_size': 128, 'max_patience': 7}. Best is trial 0 with value: 0.3628803137689829.
[I 2024-03-20 01:29:20,434] Trial 2 finished with value: 0.3860214464366436 and parameters: {'lr': 0.00012339558044188945, 'batch_size': 64, 'max_patience': 19}. Best is trial 0 with value: 0.3628803137689829.
[I 2024-03-20 01:29:52,982] Trial 3 finished with value: 0.4097594106569886 and parameters: {'lr': 0.00012479148394647308, 'batch_size': 128, 'max_patience': 11}. Best is trial 0 with value: 0.3628803137689829.
[I 2024-03-20 01:30:34,474] Trial 4 finished w

Best trial:
  Value (Best Validation Accuracy): 0.3628803137689829
  Params: 
    lr: 0.0009339960914009376
    batch_size: 256
    max_patience: 7


In [3]:
pip install optuna

  Obtaining dependency information for optuna from https://files.pythonhosted.org/packages/4c/6a/219a431aaf81b3eb3070fd2d58116baa366d3072f43bbcc87dc3495b7546/optuna-3.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for alembic>=1.5.0 from https://files.pythonhosted.org/packages/7f/50/9fb3a5c80df6eb6516693270621676980acd6d5a9a7efdbfa273f8d616c7/alembic-1.13.1-py3-none-any.whl.metadata
  Obtaining dependency information for colorlog from https://files.pythonhosted.org/packages/f3/18/3e867ab37a24fdf073c1617b9c7830e06ec270b1ea4694a624038fc40a03/colorlog-6.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for Mako from https://files.pythonhosted.org/packages/2b/8d/9f11d0b9ac521febb806e7f30dc5982d0f4f5821217712c59005fbc5c1e3/Mako-1.3.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/413.4 kB ? eta -:--:--
   ------------- -------------------------- 143.4/413.4 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 413.4/4

In [19]:

model = models.FGANet(
    num_input=data.shape[2],
    num_output=1,
    conv_channels=4,
    num_in_channels=data.shape[1])
model.eval()
# Create a dummy input with the size of [batch_size, channels, width]
# where channels is num_in_channels and width is 249
dummy_input = torch.randn(1, data.shape[1], 249)

with torch.no_grad():
    x = dummy_input
    for layer in model.features:
        x = layer(x)
        print(layer.__class__.__name__, "output shape:", x.shape)

# Flatten the output from the convolutional layers
# The view operation reshapes the output to a two-dimensional tensor where the second dimension
# is the total number of features from the convolutional output
num_dense_input = conv_output.view(conv_output.size(0), -1).size(1)

# Now num_dense_input holds the total number of features for the fully connected layer
print(num_dense_input)

Conv1d output shape: torch.Size([1, 4, 247])
ReLU output shape: torch.Size([1, 4, 247])
Dropout output shape: torch.Size([1, 4, 247])
BatchNorm1d output shape: torch.Size([1, 4, 247])
Conv1d output shape: torch.Size([1, 4, 245])
ReLU output shape: torch.Size([1, 4, 245])
Dropout output shape: torch.Size([1, 4, 245])
BatchNorm1d output shape: torch.Size([1, 4, 245])
MaxPool1d output shape: torch.Size([1, 4, 122])
Conv1d output shape: torch.Size([1, 8, 120])
ReLU output shape: torch.Size([1, 8, 120])
Dropout output shape: torch.Size([1, 8, 120])
BatchNorm1d output shape: torch.Size([1, 8, 120])
Conv1d output shape: torch.Size([1, 8, 118])
ReLU output shape: torch.Size([1, 8, 118])
Dropout output shape: torch.Size([1, 8, 118])
MaxPool1d output shape: torch.Size([1, 8, 59])
BatchNorm1d output shape: torch.Size([1, 8, 59])
Conv1d output shape: torch.Size([1, 8, 57])
ReLU output shape: torch.Size([1, 8, 57])
Dropout output shape: torch.Size([1, 8, 57])
BatchNorm1d output shape: torch.Size([1

In [10]:
data.shape[2]

249

In [3]:
model = models.FGANet(
    num_input=data.shape[2],
    num_output=2,
    conv_channels=args.num_conv,
    num_in_channels=data.shape[1],
    stride=1,)
model.eval()
dummy_input = torch.randn(1, data.shape[1], 249)
with torch.no_grad():
    x = dummy_input
    for layer in model.features:
        x = layer(x)
        print(layer.__class__.__name__, "output shape:", x.shape)

Conv1d output shape: torch.Size([1, 16, 247])
ReLU output shape: torch.Size([1, 16, 247])
Dropout output shape: torch.Size([1, 16, 247])
BatchNorm1d output shape: torch.Size([1, 16, 247])
Conv1d output shape: torch.Size([1, 16, 245])
ReLU output shape: torch.Size([1, 16, 245])
Dropout output shape: torch.Size([1, 16, 245])
BatchNorm1d output shape: torch.Size([1, 16, 245])
MaxPool1d output shape: torch.Size([1, 16, 122])
Conv1d output shape: torch.Size([1, 32, 120])
ReLU output shape: torch.Size([1, 32, 120])
Dropout output shape: torch.Size([1, 32, 120])
BatchNorm1d output shape: torch.Size([1, 32, 120])
Conv1d output shape: torch.Size([1, 32, 118])
ReLU output shape: torch.Size([1, 32, 118])
Dropout output shape: torch.Size([1, 32, 118])
MaxPool1d output shape: torch.Size([1, 32, 59])
BatchNorm1d output shape: torch.Size([1, 32, 59])
Flatten output shape: torch.Size([1, 1888])


In [4]:
args.num_conv

16

In [10]:
print(best_weights)

NameError: name 'best_weights' is not defined

model.features

In [14]:
model.features

Sequential(
  (0): Conv1d(1, 16, kernel_size=(3,), stride=(1,))
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Conv1d(16, 16, kernel_size=(3,), stride=(1,))
  (5): ReLU()
  (6): Dropout(p=0.3, inplace=False)
  (7): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (9): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
  (10): ReLU()
  (11): Dropout(p=0.3, inplace=False)
  (12): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (13): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  (14): ReLU()
  (15): Dropout(p=0.4, inplace=False)
  (16): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (18): Conv1d(32, 32, kernel_size=(3,), stride=(1,))

In [15]:
model.features[-5]

Conv1d(32, 32, kernel_size=(3,), stride=(1,))

In [5]:
final_labels.shape

(32954, 2)